https://zhuanlan.zhihu.com/p/24841366485

## basics

- cuda kernel vs. triton kernel
- KVcache
    - GQA/MQA: key-value caches are shared cross query heads
    - NSA实质的Attention计算是GQA
- notions
    - $h$: head index, $H$: the number of query heads

## algorithms

### token compression

- $\ell$: block size, $d$: stride
    - $t=9$, $\ell = 3$, $d=2$
        - $M=\lfloor\frac{t-\ell}{d}\rfloor=3$
        - [1, 2, 3], [3, 4, 5], [5, 6, 7], [7, 8, 9]
    - $\ell=d$ 时，无重叠；

### token selection

> Selection Attention才是真正体现了Sparse的精髓。


$$
p_t^{cmp}=\text{Softmax}(q_t^T\tilde K_t^{cmp})
$$

- $q_t \in(d_k, 1)$；$\tilde K_t^{cmp}\in (d_k, M)$
- $p_t^{cmp}=(1, M)$
- $\ell'$：也是 (selection) block size

## kernel 优化

- 常规：我们需要将单头KV，复制成与Q头数相对应的KV，送到SRAM计算注意力
- NSA：单头KV送到SRAM，这里不需要复制多头。